In [ ]:
import os
import pandas as pd
import csv
import re

import nltk
import requests
import numpy as np
nltk.download('punkt')
nltk.download("stopwords")
import copy

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation

!pip install pandarallel
from pandarallel import pandarallel

from google.colab import drive
drive.mount('/content/drive')


%cd /content/drive/My\ Drive/SNA2020/Project

df_bots_pymorphy=pd.read_csv('./bots_pymorphy.tsv',sep="\t",header=0)
df_bots_pymorphy.head()

df_politics_pymorphy=pd.read_csv('./politics_pymorphy.tsv',sep="\t",header=0)
df_politics_pymorphy.head()

url_stopwords_ru = "https://raw.githubusercontent.com/stopwords-iso/stopwords-ru/master/stopwords-ru.txt"

badwords = [
  u'я', u'а', u'да', u'но', u'тебе', u'мне', u'ты', u'и', u'у', u'на', u'ща', u'ага',
  u'так', u'там', u'какие', u'который', u'какая', u'туда', u'давай', u'короче', u'кажется', u'вообще',
  u'ну', u'не', u'чет', u'неа', u'свои', u'наше', u'хотя', u'такое', u'например', u'кароч', u'как-то',
  u'нам', u'хм', u'всем', u'нет', u'да', u'оно', u'своем', u'про', u'вы', u'м', u'тд',
  u'вся', u'кто-то', u'что-то', u'вам', u'это', u'эта', u'эти', u'этот', u'прям', u'либо', u'как', u'мы',
  u'просто', u'блин', u'очень', u'самые', u'твоем', u'ваша', u'кстати', u'вроде', u'типа', u'пока', u'ок'
]


def get_text(url, encoding='utf-8', to_lower=True):
    url = str(url)
    if url.startswith('http'):
        r = requests.get(url)
        if not r.ok:
            r.raise_for_status()
        return r.text.lower() if to_lower else r.text
    elif os.path.exists(url):
        with open(url, encoding=encoding) as f:
            return f.read().lower() if to_lower else f.read()
    else:
        raise Exception('parameter [url] can be either URL or a filename')

github_stopwords = get_text(url_stopwords_ru).splitlines()

def preprocess_text_stopwords(text, setting="nltk-stopwords"):
    # print(text)
    tokens = text.split()
    if setting == "nltk-stopwords":
      stopwords_for_delete = stopwords.words("russian")
    elif setting == "github-stopwords":
      stopwords_for_delete = github_stopwords
    elif setting == "badwords":
      stopwords_for_delete = badwords
    else:
       raise Exception('parameter setting should be fill')
    
    tokens = [token for token in tokens if token not in stopwords_for_delete \
              and len(token) > 2 \
              and token != " " \
              and token.strip() not in punctuation]
    tokens = " ".join(tokens)
    return tokens

pandarallel.initialize()

def make_output_for_one_stopword_list(base_df, name_of_people, name_of_lemm, name_of_stopword):
  # after_stopwords = base_df.copy(deep=True)
  after_stopwords = copy.deepcopy(base_df)
  after_stopwords[name_of_lemm + '_' + name_of_stopword]=base_df[name_of_lemm].parallel_apply(preprocess_text_stopwords,setting=name_of_stopword)
  after_stopwords[name_of_lemm + '_' + name_of_stopword].replace('', np.nan, inplace=True)
  after_stopwords = after_stopwords.dropna()
  after_stopwords.to_csv(name_of_people + '_' + name_of_lemm + '_' + name_of_stopword + '.tsv',sep='\t', quoting=csv.QUOTE_NONE)
  return after_stopwords

def make_output_for_many_stopword_lists(base_df, name_of_people, name_of_lemm, name_of_stopword, name_of_used_stopword):
  print(type(base_df))
  # after_stopwords = base_df.copy(deep=True)
  after_stopwords = copy.deepcopy(base_df)
  after_stopwords[name_of_lemm + '_' + name_of_stopword + '_' + name_of_used_stopword]=base_df[name_of_lemm + '_' + name_of_used_stopword].parallel_apply(preprocess_text_stopwords,setting=name_of_stopword)
  after_stopwords[name_of_lemm + '_' + name_of_stopword + '_' + name_of_used_stopword].replace('', np.nan, inplace=True)
  after_stopwords = after_stopwords.dropna()
  after_stopwords.to_csv(name_of_people + '_' + name_of_lemm + '_' + name_of_stopword + '_' + name_of_used_stopword + '.tsv',sep='\t', quoting=csv.QUOTE_NONE)
  return after_stopwords


def make_output_with_stopwords(base_df, name_of_people, name_of_lemm):
  base_df = base_df.dropna()
  
  nltk_stopwords = make_output_for_one_stopword_list(base_df, name_of_people, name_of_lemm, "nltk-stopwords")
  github_stopwords = make_output_for_one_stopword_list(base_df, name_of_people, name_of_lemm, "github-stopwords")
  badwords_stopwords = make_output_for_one_stopword_list(base_df, name_of_people, name_of_lemm, "badwords")

  print('end_first_part')

  nltk_stopwords_github_stopwords = make_output_for_many_stopword_lists(github_stopwords, name_of_people, name_of_lemm, "nltk-stopwords", "github-stopwords")
  print("nltk_stopwords_github_stopwords")
  nltk_stopwords_badwords = make_output_for_many_stopword_lists(badwords_stopwords, name_of_people, name_of_lemm, "nltk-stopwords", "badwords")

  github_stopwords_badwords = make_output_for_many_stopword_lists(badwords_stopwords, name_of_people, name_of_lemm, "github-stopwords", "badwords")

  nltk_stopwords_github_stopwords_badwords = make_output_for_many_stopword_lists(github_stopwords_badwords, name_of_people, name_of_lemm, "nltk-stopwords", "github-stopwords_badwords")


# make_output_with_stopwords(df_bots_pymorphy, "bots", "pymorphy")

make_output_with_stopwords(df_politics_pymorphy, "politics", "pymorphy")


